In [1]:
import pyomo.environ as pyo
from solar_installations import INSTALLATION_SIZES

In [2]:
model = pyo.ConcreteModel()
model.SOLAR_SIZES = pyo.Set(initialize=INSTALLATION_SIZES.keys())
model.solar_size_flags = pyo.Var(model.SOLAR_SIZES, within=pyo.Binary)
model.installed_solar = pyo.Var(within=pyo.NonNegativeReals)


def installed_solar_constraint(model):
    sum_term = sum(model.solar_size_flags[i] for i in model.SOLAR_SIZES)
    return model.installed_solar == sum_term


model.consumption_constraint = pyo.Constraint(rule=installed_solar_constraint)
model.sos1_constraint = pyo.SOSConstraint(var=model.solar_size_flags, sos=1)


def required_power(model):
    return model.installed_solar >= 6


model.required_power_constraint = pyo.Constraint(rule=required_power)

"""
def solar_cost(model):
    return model.solar_cost == sum(
        INSTALLATION_SIZES[size] * model.solar_size_flags[size]
        for size in model.SOLAR_SIZES
    )


model.solar_cost_constraint = pyo.Constraint(rule=solar_cost)
"""
model.objective = pyo.Objective(expr=model.installed_solar, sense=pyo.maximize)
optimizer = pyo.SolverFactory("gurobi")
optimizer.solve(model)
print(model.display())

model.name="unknown";
    - termination condition: infeasibleOrUnbounded
    - message from solver: Problem proven to be infeasible or unbounded.
Model unknown

  Variables:
    solar_size_flags : Size=6, Index=SOLAR_SIZES
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          3 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : False :  True : Binary
         10 :     0 :  None :     1 : False :  True : Binary
         12 :     0 :  None :     1 : False :  True : Binary
    installed_solar : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

  Objectives:
    objective : Size=1, Index=None, Active=True
ERROR: evaluating object as numeric value: objective
        (object: <class 'pyomo.core.base.objective.ScalarOb